In [28]:
import numpy as np

In [29]:
import matplotlib.pyplot as plt

import matplotlib.animation as animation 

In [30]:
from IPython import display 

In [31]:
import sys

sys.path.append('../../..')

In [32]:
from lib_spline.catmullrom import CatmullRom
from lib_spline.piecewise import Piecewise
from lib_spline.grid import alpha_param

In [33]:
from lib_param.interface.expression import Expression

In [34]:
points = np.array([
    
    (-1, -0.5, 1.0),
    (0, 2.3, 0.1),
    (0.15, 3.1, 0.5),
    (4, 1.3, -1.5)
])

In [35]:
# init data

new_grid = alpha_param(0.0, points, closed = True)
catmull = CatmullRom( points, new_grid, closed = True )
tetha = np.linspace(catmull.grid[0], catmull.grid[-1], 100)
render = np.array([catmull.evaluate(t) for t in tetha])

In [36]:
e = Expression (catmull)

torsion = np.array( [ e.torsion(t1) for t1 in tetha ] )

start = np.array( [ np.array(e.normal(t1)) * torsion[idx] for idx, t1 in enumerate(tetha) ] )

In [37]:
from matplotlib.colors import LinearSegmentedColormap

from mpl_toolkits.mplot3d.art3d import Line3DCollection

In [38]:
cmap = plt.cm.get_cmap('jet')  

/tmp/ipykernel_10231/3779237694.py:1: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('jet')


In [39]:
def lerp(xs, ts, t):

    x_str, x_end = xs

    t_str, t_end = ts

    t_map = (t - t_str) / (t_end - t_str)

    return (1 - t_map) * x_str + t_map * x_end

In [42]:
fig = plt.figure(figsize=(15, 5))

axl = fig.add_subplot(1, 2, 1, projection='3d')
axr = fig.add_subplot(1, 2, 2)

# axes prop - right

axl.set_xlim3d([-3, 6])
axl.set_xlabel('X')

axl.set_ylim3d([-2, 4])
axl.set_ylabel('Y')

axl.set_zlim3d([-3, 6])
axl.set_zlabel('Z')

# axes prop - left

axr.set_ylim([-20, 20])

# left
sm = plt.cm.ScalarMappable(cmap=cmap) 
sm.set_array(torsion) 
fig.colorbar(sm, ax=axl, shrink=0.85) 

lc = Line3DCollection([], cmap=cmap, linewidth = 2.5)
axl.add_collection(lc)

# right
lineR, = axr.plot(tetha, torsion)

max_frames = 100

def init ():

    pass

def animate(frame_number):

    mid_frame = (max_frames / 2)

    if (frame_number < mid_frame):
        interpolator  = frame_number / max_frames
    else:
        interpolator  = 1.0 - (frame_number / max_frames)

    alpha = lerp([-0.5, 1], [0, 0.5], interpolator)

    new_grid = alpha_param(alpha, points, closed = True)
    catmull = CatmullRom( points, new_grid, closed = True )
    tetha = np.linspace(catmull.grid[0], catmull.grid[-1], 100)
    render = np.array([catmull.evaluate(t) for t in tetha])

    e = Expression (catmull)

    points_1 = render.reshape(-1,1,3)
    segments = np.concatenate([points_1[:-1], points_1[1:]], axis=1)   
    torsion = np.array( [ e.torsion(t1) for t1 in tetha ] )

    lc.set_segments(segments)
    lc.set_array(torsion)

    # torsion plot -> right:

    lineR.set_ydata(torsion)

    return lc, lineR

anim = animation.FuncAnimation(fig, animate, frames =max_frames, interval=20, blit=False)

fig.suptitle('Alpha Parametrization - Torsion', fontsize=14) 
  
video = anim.to_html5_video() 
  
html = display.HTML(video) 
  
display.display(html) 

plt.close() 